### 모델 파인튜닝

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import torch

# # Mount Google Drive (for Colab users)
# from google.colab import drive
# drive.mount('/content/drive')

# Load the dataset
file_path = "/content/최종_df.csv"
data = pd.read_csv(file_path)

# Split data into train and validation
train_data = data.sample(frac=0.8, random_state=42)
val_data = data.drop(train_data.index)

# Convert to Hugging Face Dataset
def prepare_data(data):
    return Dataset.from_pandas(data[['input', 'output']])

train_dataset = prepare_data(train_data)
val_dataset = prepare_data(val_data)

# Load tokenizer and model
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v2")
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-base-v2")

# Tokenize the data
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input'], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples['output'], max_length=512, truncation=True, padding="max_length").input_ids
    model_inputs['labels'] = labels
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./kobart_results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./kobart_logs',
    logging_steps=10,
    save_steps=500,
    warmup_steps=500,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("./kobart-finetuned")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

...

Map:   0%|          | 0/798 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-c89b50a98989>:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,3.596300,3.219708
2,0.494600,0.436530
3,0.299800,0.291311


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


### 출력 테스트

In [4]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# Load the tokenizer and the trained model
model_path = "/content/kobart-finetuned"
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v2")
model = BartForConditionalGeneration.from_pretrained(model_path)

# Input text for testing
test_input = """
“친하다고 해야 하나, 지긋지긋한 인연이라 해야 하나.”  계단을 내려가 신발장에서 신발을 꺼내 신고 밖으로 나왔다. 아침나절보다 따뜻해져 미지근한 봄바람이 꽃향기를 풍기며 뺨을 간질이고 지나갔다.  “넌 동아리 가입 안 했어?”  “응, 전에 좀 다쳤거든. 일상생활에는 지장이 없는데, 격렬한 운동은 하면 안 돼.”  “그랬구나. 큰일 날 뻔했네.”  계속 내 옆에서 어깨를 마주하고 걷던 네가 눈을 내리깔며 말했다. 기다란 속눈썹이 예쁘게 깜빡여서 절로 눈이 갔다.  “넌 집이 어디야?”  말머리를 돌리려고 물었다. 좋을 것도, 나쁠 것도 없는 시시한 질문이다.  “요 옆에 시노노메초. 버스 탈 때도 있고 걸어서 갈 때도 있는데, 걸어가면 30분쯤 걸려.”  “난 시라아이초. 우리 동네랑 가깝네.”  뜻밖의 정보를 알아냈다. 별것 아닌데도 내심 기뻤다.  “넌 걸어 다녀?”  “응. 걸으면 20분 좀 안 걸리는데, 시간 없을 때는 자전거도 타고.”
"""

# Encode the input
inputs = tokenizer(test_input, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

# Generate predictions
output_ids = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=512,
    num_beams=4,
    length_penalty=1.0,
    early_stopping=True
)

# Decode the generated text
generated_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated Output:")
print(generated_output)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Generated Output:
[location] 산봉우리

[caption] 봄바람이 봄바람을 타고 산봉우리와 산봉우리 사이를 거닐며 걷기 시작했다.

[dialogues]
  [
  [speaker] 산봉우리
  [dialogue] "넌 동아리 가입 안 했어?"
  ]


In [8]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import re

# 모델과 토크나이저 로드
model_path = "/content/kobart-finetuned"
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v2")
model = BartForConditionalGeneration.from_pretrained(model_path)

def slice_and_process_text(input_file_path, output_file_path, slice_num=20000, chunk_size=500):
    """
    텍스트를 지정된 길이로 자르고, 모델을 통해 처리한 후, 결과를 저장합니다.
    """
    # 텍스트 파일 읽기
    with open(input_file_path, 'r', encoding='utf-8') as f:
        input_txt = f.read()

    # 줄바꿈 제거 및 텍스트 자르기
    input_txt = input_txt.replace('\n', ' ')
    input_txt = input_txt[:slice_num]  # 지정된 크기만큼 자르기

    # 문장 끝 패턴 정의 및 분리
    sentence_end_pattern = re.compile(r'([.!?]["”]?\s)')  # 문장 끝 패턴
    sentences = sentence_end_pattern.split(input_txt)

    # 문장을 청크로 나누기
    chunks = []
    current_chunk = ""

    for i in range(0, len(sentences) - 1, 2):
        sentence = sentences[i] + sentences[i + 1]
        if len(current_chunk) + len(sentence) > chunk_size:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
        else:
            current_chunk += sentence

    if current_chunk:
        chunks.append(current_chunk.strip())

    # 모델을 통해 청크 처리 및 결과 저장
    with open(output_file_path, 'w', encoding='utf-8') as f:
        for chunk in chunks:
            # 모델 입력 생성
            inputs = tokenizer(chunk, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

            # 모델 출력 생성
            output_ids = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=512,
                num_beams=4,
                length_penalty=1.0,
                early_stopping=True
            )

            # 디코딩 및 저장
            generated_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            f.write(generated_output + '\n\n')  # 두 줄 공백으로 구분하여 저장

    print(f"모델 출력을 저장했습니다: {output_file_path}")

# 사용 예시
input_path = f"/content/화산귀환 (비가) (Z-Library)_원문.txt"
output_path = f"/content/화산귀환_output.txt"
slice_and_process_text(input_path, output_path)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


모델 출력을 저장했습니다: /content/화산귀환_output.txt


In [9]:
import re
import json

def parse_text_file_to_json(input_file_path, output_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # 각 컷은 두 줄 공백으로 구분
    sections = text.strip().split("\n\n")

    # JSON 형식으로 변환할 데이터를 저장할 리스트
    json_data = []

    # 각 섹션을 순회하며 데이터를 추출
    for section in sections:
        location_match = re.search(r'\[location\]\s*"(.*?)"', section)
        caption_match = re.search(r'\[caption\]\s*"(.*?)"', section)
        dialogues_matches = re.findall(r'\[dialogue\]\s*"(.*?)"\s*\[speaker\]\s*"(.*?)"', section)

        json_entry = {
            "location": location_match.group(1) if location_match else None,
            "caption": caption_match.group(1) if caption_match else None,
            "dialogues": [
                {"dialogue": dialogue, "speaker": speaker} for dialogue, speaker in dialogues_matches
            ]
        }

        # 유효한 데이터만 추가
        if json_entry["location"] or json_entry["caption"] or json_entry["dialogues"]:
            json_data.append(json_entry)

    # JSON 데이터를 파일로 저장
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

    print(f"JSON 파일이 저장되었습니다: {output_file_path}")

# 입력 및 출력 파일 경로
input_file_path = "/content/할아버지의 달콤한 유산_output.txt"
output_file_path = "/content/할아버지의 달콤한 유산_output.json"

# 변환 실행
parse_text_file_to_json(input_file_path, output_file_path)

JSON 파일이 저장되었습니다: /content/할아버지의 달콤한 유산_output.json


In [19]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import re

# 모델과 토크나이저 로드
model_path = "/content/kobart-finetuned"
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v2")
model = BartForConditionalGeneration.from_pretrained(model_path)

def slice_and_process_text(input_file_path, slice_num=20000, chunk_size=500):
    """
    텍스트를 지정된 길이로 자르고, 사용자 입력을 받을 때마다 모델 결과를 출력합니다.
    """
    # 텍스트 파일 읽기
    with open(input_file_path, 'r', encoding='utf-8') as f:
        input_txt = f.read()

    # 줄바꿈 제거 및 텍스트 자르기
    input_txt = input_txt.replace('\n', ' ')
    input_txt = input_txt[:slice_num]  # 지정된 크기만큼 자르기

    # 문장 끝 패턴 정의 및 분리
    sentence_end_pattern = re.compile(r'([.!?]["”]?\s)')  # 문장 끝 패턴
    sentences = sentence_end_pattern.split(input_txt)

    # 문장을 청크로 나누기
    chunks = []
    current_chunk = ""

    for i in range(0, len(sentences) - 1, 2):
        sentence = sentences[i] + sentences[i + 1]
        if len(current_chunk) + len(sentence) > chunk_size:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
        else:
            current_chunk += sentence

    if current_chunk:
        chunks.append(current_chunk.strip())

    # 청크별로 모델을 통해 처리하고 결과를 출력
    for idx, chunk in enumerate(chunks):
        # 모델 입력 생성
        inputs = tokenizer(chunk, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

        # 모델 출력 생성
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=512,
            num_beams=4,
            length_penalty=1.0,
            early_stopping=True,
            # no_repeat_ngram_size = 3,
            # repetition_penalty = 1.5
        )

        # 디코딩 및 출력
        generated_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        print(f"청크 {idx + 1} 출력:\n{generated_output}\n")

        # 사용자 입력 대기
        user_input = input("출력을 확인한 후 '완료'를 입력하세요: ")
        if user_input.strip().lower() != "완료":
            print("작업을 중단합니다.")
            break

# 사용 예시
input_path = f"/content/할아버지의 달콤한 유산_output.txt"
slice_and_process_text(input_path)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


청크 1 출력:
[location] "학교 연극 동아리 무대"
[caption] "장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원 장팅원

출력을 확인한 후 '완료'를 입력하세요: 완료
청크 2 출력:
[location] "할머니의 거처"
[caption] "할머니의 거처"
[dialogue] "할머니의 거처"
[speaker] "할머니"
[dialogue] "할머니의 거처"
[speaker] "할머니"
[dialogue] "너는 내 얘기를 귓등으로도 안 듣는 게냐!" [speaker] "할머니"
[dialogue] "할머니의 거처"
[speaker] "할머니"
[dialogue] "할머니의 거처"
[speaker] "할머니"
]



KeyboardInterrupt: Interrupted by user